In [2]:
import requests
from time import sleep
import time
import pandas as pd
import numpy as np
import json
from flatten_json import flatten

In [12]:
#Compiles and cleans PAIRbulk data from file
#Returns patent Nx7 df for patents with maintenance fees

from os import listdir
from os.path import isfile, join
#NOTE
#PAIRBULKDATA CAN BE DOWNLOADED VIA 
#https://pairbulkdata.uspto.gov/#/search?fq=appStatus%3APatent%20Expired%20Due%20to%20NonPayment%20of%20Maintenance%20Fees%20Under%2037%20CFR%201.362&sort=applId%20asc
def pairbulkdata_cleaner(filepath):
    start_time_pair = time.time()
    onlyfiles = [f for f in listdir(filepath) if isfile(join(filepath, f))]
    onlyfiles.sort()
    df = {}
    counter = 0
    for fname in onlyfiles:
        path = 'pairbulkdata/' + fname
        #path = 'pairbulkdata/1970.json'
        json1_file = open(path)
        json1_str = json1_file.read()
        json_file = json.loads(json1_str)
        json_dict = json_file["PatentBulkData"]
        dic_flattened = (flatten(d) for d in json_dict)
        inter_df = pd.DataFrame(dic_flattened)
        #Columns to keep; expiration date, entity size, application date, title, grant date, patent #, uspto link
        inter_df = inter_df[['applicationDataOrProsecutionHistoryDataOrPatentTermData_0_applicationStatusDate',
                             'applicationDataOrProsecutionHistoryDataOrPatentTermData_0_businessEntityStatusCategory',
                             'applicationDataOrProsecutionHistoryDataOrPatentTermData_0_filingDate',
                             'applicationDataOrProsecutionHistoryDataOrPatentTermData_0_inventionTitle_content_0',
                             'applicationDataOrProsecutionHistoryDataOrPatentTermData_0_patentGrantIdentification_grantDate',
                             'applicationDataOrProsecutionHistoryDataOrPatentTermData_0_patentGrantIdentification_patentNumber',
                             'applicationDataOrProsecutionHistoryDataOrPatentTermData_2_grantPublication_webURI']]
        df[fname] = inter_df
        counter += 1
        #===============HYPERPARAMETER====================
        #CONTROLS NUMBER OF PAIRBULK DATA FILES TO READ
        #break for my poor computer
        if counter == 5:
            break

    pairbulk_df = pd.concat(df.values(), ignore_index=True)
    export_path = "pairbulkdata/"
    export_fname = "pairbulk_clean.csv"
    pairbulk_df.to_csv((export_path+export_fname))
    print("============Pairbulk Data Set Processed============")
    print("Run time ====== %s seconds=====" % (time.time()-start_time_pair) )
    print("File located at ->",export_path," with filename ->",export_fname)
    return(pairbulk_df)

In [10]:
def patentsviewCaller(pairbulk_clean):
    #if file exists in LOC read from there
    from pathlib import Path
    import csv
    start_time_pvcaller = time.time()
    my_file = Path("pairbulkdata/")
    if my_file.is_file():
        with open('pairbulk_clean.csv') as csvfile:
            pairbulk_clean = csv.reader(csvfile)
    #Extract array of patents
    patents_array = pairbulk_clean['applicationDataOrProsecutionHistoryDataOrPatentTermData_0_patentGrantIdentification_patentNumber'].str.strip('[]')
    #TEST SET
    #patents_array = [5641936,564193126,224338394,4899874,5623524]
    
    #Call PatentsView API
    #==============API Fields===============
    #http://www.patentsview.org/api/patent.html
    patent = '"patent_number"'
    ##Assignee Data
    assignee_id = '"assignee_id"'
    assignee_org = '"assignee_organization"'
    assignee_inventors = '"assignee_total_num_inventors"'
    assignee_patents = '"assignee_total_num_patents"'
    assignee_type = '"assignee_type"'
    ##Classification Data
    cpc_seq = '"cpc_sequence"'
    cpc = '"cpc_group_id"'
    cpc_subgroup = '"cpc_subgroup_id"'
    cpc_assignees = '"cpc_total_num_assignees"'
    cpc_inventors = '"cpc_total_num_inventors"'
    cpc_patents = '"cpc_total_num_patents"'
    ##Identifiers
    examiner = '"examiner_id"'
    inventor = '"patent_firstnamed_inventor_id"'
    lawyer = '"lawyer_id"'
    ##Patent Metrics
    avg_proc_time = '"patent_average_processing_time"'
    forward_cites_us = '"patent_num_cited_by_us_patents"'
    back_cites_us = '"patent_num_us_patent_citations"'
    back_cites_foreign = '"patent_num_foreign_citations"'
    claims = '"patent_num_claims"'
    proc_time = '"patent_processing_time"'

    #Assemble string
    #Note: Can be improved, save ids to list, iteratively generate  request string
    request_string_prefix = 'http://www.patentsview.org/api/patents/query?q={"patent_number":"'
    request_string_suffix = ('"}&f=['+patent","+assignee_id+","+assignee_org+","+assignee_inventors+","+assignee_patents+","+
    assignee_type+","+cpc_seq+","+cpc+","+cpc_subgroup+","+cpc_assignees+","+cpc_inventors+","+cpc_patents+
    ","+examiner+","+inventor+","+lawyer+","+avg_proc_time+","+forward_cites_us+","+back_cites_us+","+back_cites_foreign+
    ","+claims+","+proc_time+']')
    temp_list = []
    #Run Request on Patents Array
    for patent in patents_array:
        request_string = request_string_prefix+str(patent)+request_string_suffix
        #print(request_string)
    #Check Data integrity
    #NOTE: US Patents take format of #######, if item is >= 10000000 is not valid patent
        if int(patent) <= 9999999:
            request = requests.get(request_string)
            status = request.status_code
            data = request.json()
            print('Fetching %s...status code %d: '%(patent, status))
        else:
            data = {}
            print('Unable to fetch %s...non US patent Format'%(patent))
        sleep(0.2)
        temp_list.append(data)

    #Flatten list of dicts
    temp_list = (flatten(d) for d in temp_list)
    #Place flattend list in DF
    patentsview_clean = pd.DataFrame(temp_list)
    #Export
    export_path = 'patentsviewdata/'
    export_fname = 'test.csv'
    patentsview_clean.to_csv((export_path+export_fname))
    
    print("============Patentsview Data Set Processed============")
    print("Run time ====== %s seconds=====" % (time.time()-start_time_pvcaller) )
    print("File located at ->",export_path," with filename ->",export_fname)
    return(patentsview_clean)

In [5]:
def merge_and_clean(patentsview_clean, pairbulk_clean):
    #Join
    shiny_dataset = pairbulk_clean.join(patentsview_clean)
    shiny_dataset = shiny_dataset.drop('count',1)
    shiny_dataset = shiny_dataset.drop('total_patent_count',1)

    #Export Master_DF to csv and npy
    filepath = "master_data/"
    filename =  "patent_master.csv"
    filename_numpy = "patent_mt_master.npy"
    shiny_dataset_np = shiny_dataset.as_matrix()
    shiny_dataset.to_csv((filepath+filename))
    np.save((filepath+filename),shiny_dataset_np)

    print("============Master Data Set Processed============")
    print("Run time ====== %s seconds=====" % (time.time()-start_time) ) 
    print("CSV file located at ->", filepath, " with filename ->",filename)
    print("Numpy Matrix located at ->", filepath, " with filename ->",filename_numpy,"'")

    return(print("=====ROUTINE COMPLETED======"))

In [11]:
#===============MAIN ROUTINE=====================
#========================================
start_time = time.time()

#Run PAIR BULK DATA CLEANER
filepath = "pairbulkdata"
pairbulk_clean = pairbulkdata_cleaner(filepath)
print(pairbulk_clean.shape)

#call and clean patentsview data
patentsview_clean = patentsviewCaller(pairbulk_clean)
print(patentsview_clean.shape)

#Merge dataframes clean and run exports
merge_and_clean(patentsview_clean,pairbulk_clean)


============Pairbulk Data Set Processed============
Run time ====== 8.280879497528076 seconds=====
File located at -> pairbulkdata/  with filename -> pairbulk_clean.csv
(18084, 7)
Fetching 5641936...status code 200: 
Fetching 5623524...status code 200: 
Fetching 4076779...status code 200: 
Fetching 5706850...status code 200: 
Fetching 4303653...status code 200: 
Fetching 4423428...status code 200: 
Fetching 4423892...status code 200: 
Fetching 4350887...status code 200: 
Fetching 4381041...status code 200: 
Fetching 4351658...status code 200: 
Fetching 4319972...status code 200: 
Fetching 4350890...status code 200: 
Fetching 4348357...status code 200: 
Fetching 4365467...status code 200: 
Fetching 4395172...status code 200: 
Fetching 4356380...status code 200: 
Fetching 4398532...status code 200: 
Fetching 4378186...status code 200: 
Fetching 4352497...status code 200: 
Fetching 4367688...status code 200: 
Fetching 4322608...status code 200: 
Fetching 4367042...status code 200: 
Fetchi

Fetching 4363780...status code 200: 
Fetching 4375620...status code 200: 
Fetching 4389062...status code 200: 
Fetching 4347577...status code 200: 
Fetching 4349884...status code 200: 
Fetching 4354242...status code 200: 
Fetching 4321416...status code 200: 
Fetching 4310377...status code 200: 
Fetching 4345000...status code 200: 
Fetching 4372382...status code 200: 
Fetching 4397052...status code 200: 
Fetching 4335737...status code 200: 
Fetching 4337957...status code 200: 
Fetching 4350445...status code 200: 
Fetching 4410401...status code 200: 
Fetching 4364443...status code 200: 
Fetching 4353349...status code 200: 
Fetching 4362425...status code 200: 
Fetching 4325504...status code 200: 
Fetching 4350080...status code 200: 
Fetching 4392261...status code 200: 
Fetching 4385206...status code 200: 
Fetching 4305243...status code 200: 
Fetching 4374350...status code 200: 
Fetching 4367181...status code 200: 
Fetching 4375137...status code 200: 
Fetching 4380131...status code 200: 
F

Fetching 4376911...status code 200: 
Fetching 4385506...status code 200: 
Fetching 4339036...status code 200: 
Fetching 4389915...status code 200: 
Fetching 4365973...status code 200: 
Fetching 4363316...status code 200: 
Fetching 4338374...status code 200: 
Fetching 4338373...status code 200: 
Fetching 4320557...status code 200: 
Fetching 4423212...status code 200: 
Fetching 4302603...status code 200: 
Fetching 4334099...status code 200: 
Fetching 4330547...status code 200: 
Fetching 4330955...status code 200: 
Fetching 4378950...status code 200: 
Fetching 4306012...status code 200: 
Fetching 4365083...status code 200: 
Fetching 4344484...status code 200: 
Fetching 4403577...status code 200: 
Fetching 4359933...status code 200: 
Fetching 4350085...status code 200: 
Fetching 4384152...status code 200: 
Fetching 4350602...status code 200: 
Fetching 4373685...status code 200: 
Fetching 4322401...status code 200: 
Fetching 4338478...status code 200: 
Fetching 4337122...status code 200: 
F

Fetching 4429203...status code 200: 
Fetching 4360661...status code 200: 
Fetching 4367468...status code 200: 
Fetching 4372192...status code 200: 
Fetching 4353518...status code 200: 
Fetching 4374112...status code 200: 
Fetching 4379598...status code 200: 
Fetching 4310593...status code 200: 
Fetching 4378421...status code 200: 
Fetching 4350116...status code 200: 
Fetching 4352626...status code 200: 
Fetching 4322769...status code 200: 
Fetching 4374383...status code 200: 
Fetching 4389449...status code 200: 
Fetching 4372672...status code 200: 
Fetching 4375538...status code 200: 
Fetching 4366652...status code 200: 
Fetching 4312639...status code 200: 
Fetching 4387213...status code 200: 
Fetching 4354473...status code 200: 
Fetching 4342583...status code 200: 
Fetching 4389214...status code 200: 
Fetching 4325932...status code 200: 
Fetching 4386307...status code 200: 
Fetching 4357472...status code 200: 
Fetching 4377292...status code 200: 
Fetching 4342876...status code 200: 
F

Fetching 4327160...status code 200: 
Fetching 4312715...status code 200: 
Fetching 4391974...status code 200: 
Fetching 4349260...status code 200: 
Fetching 4381116...status code 200: 
Fetching 4318326...status code 200: 
Fetching 4350069...status code 200: 
Fetching 4358981...status code 200: 
Fetching 4361966...status code 200: 
Fetching 4359089...status code 200: 
Fetching 4362965...status code 200: 
Fetching 4380749...status code 200: 
Fetching 4345236...status code 200: 
Fetching 4318911...status code 200: 
Fetching 4381853...status code 200: 
Fetching 4364200...status code 200: 
Fetching 4389545...status code 200: 
Fetching 4341799...status code 200: 
Fetching 4363577...status code 200: 
Fetching 4341008...status code 200: 
Fetching 4387850...status code 200: 
Fetching 4375061...status code 200: 
Fetching 4396629...status code 200: 
Fetching 4387529...status code 200: 
Fetching 4369943...status code 200: 
Fetching 4328914...status code 200: 
Fetching 4346467...status code 200: 
F

...

Fetching 4450400...status code 200: 
Fetching 4433827...status code 200: 
Fetching 4379788...status code 200: 
Fetching 4413262...status code 200: 
Fetching 4392307...status code 200: 
Fetching 4541028...status code 200: 
Fetching 4438070...status code 200: 
Fetching 4441733...status code 200: 
Fetching 4406207...status code 200: 
Fetching 4421992...status code 200: 
Fetching 4444183...status code 200: 
Fetching 4360200...status code 200: 
Fetching 4388949...status code 200: 
Fetching 4380596...status code 200: 
Fetching 4392887...status code 200: 
Fetching 4424851...status code 200: 
Fetching 4440167...status code 200: 
Fetching 4370082...status code 200: 
Fetching 4417552...status code 200: 
Fetching 4431530...status code 200: 
Fetching 4374194...status code 200: 
Fetching 4436330...status code 200: 
Fetching 4394368...status code 200: 
Fetching 4400973...status code 200: 
Fetching 4437610...status code 200: 
Fetching 4630583...status code 200: 
Fetching 4402424...status code 200: 
F

Fetching 4395370...status code 200: 
Fetching 4480646...status code 200: 
Fetching 4437692...status code 200: 
Fetching 4440338...status code 200: 
Fetching 4460078...status code 200: 
Fetching 4410166...status code 200: 
Fetching 4377417...status code 200: 
Fetching 4458660...status code 200: 
Fetching 4386774...status code 200: 
Fetching 4380305...status code 200: 
Fetching 4400201...status code 200: 
Fetching 4382793...status code 200: 
Fetching 4468778...status code 200: 
Fetching 4411358...status code 200: 
Fetching 4354019...status code 200: 
Fetching 4507638...status code 200: 
Fetching 4388253...status code 200: 
Fetching 4366808...status code 200: 
Fetching 4445553...status code 200: 
Fetching 4403714...status code 200: 
Fetching 4457123...status code 200: 
Fetching 4443805...status code 200: 
Fetching 4472159...status code 200: 
Fetching 4413969...status code 200: 
Fetching 4436968...status code 200: 
Fetching 4396451...status code 200: 
Fetching 4363652...status code 200: 
F

Fetching 4381574...status code 200: 
Fetching 4394398...status code 200: 
Fetching 4382956...status code 200: 
Fetching 4426706...status code 200: 
Fetching 4382851...status code 200: 
Fetching 4461545...status code 200: 
Fetching 4429526...status code 200: 
Fetching 4419598...status code 200: 
Fetching 4561453...status code 200: 
Fetching 4456113...status code 200: 
Fetching 4401582...status code 200: 
Fetching 4597497...status code 200: 
Fetching 4516116...status code 200: 
Fetching 4396108...status code 200: 
Fetching 4391636...status code 200: 
Fetching 4473944...status code 200: 
Fetching 4380495...status code 200: 
Fetching 4389049...status code 200: 
Fetching 4402177...status code 200: 
Fetching 4404988...status code 200: 
Fetching 4501369...status code 200: 
Fetching 4458546...status code 200: 
Fetching 4388449...status code 200: 
Fetching 4399871...status code 200: 
Fetching 4422493...status code 200: 
Fetching 4381971...status code 200: 
Fetching 4439728...status code 200: 
F

Fetching 4380683...status code 200: 
Fetching 4387210...status code 200: 
Fetching 4406316...status code 200: 
Fetching 4413622...status code 200: 
Fetching 4398136...status code 200: 
Fetching 4396682...status code 200: 
Fetching 4421100...status code 200: 
Fetching 4429888...status code 200: 
Fetching 4435954...status code 200: 
Fetching 4459943...status code 200: 
Fetching 4401060...status code 200: 
Fetching 4371665...status code 200: 
Fetching 4375349...status code 200: 
Fetching 4509380...status code 200: 
Fetching 4546652...status code 200: 
Fetching 4368345...status code 200: 
Fetching 4397539...status code 200: 
Fetching 4411946...status code 200: 
Fetching 4396640...status code 200: 
Fetching 4464424...status code 200: 
Fetching 4377879...status code 200: 
Fetching 4453384...status code 200: 
Fetching 4449868...status code 200: 
Fetching 4391061...status code 200: 
Fetching 4433777...status code 200: 
Fetching 4400104...status code 200: 
Fetching 4402620...status code 200: 
F

Fetching 4461527...status code 200: 
Fetching 4516580...status code 200: 
Fetching 4459621...status code 200: 
Fetching 4379817...status code 200: 
Fetching 4459442...status code 200: 
Fetching 4476943...status code 200: 
Fetching 4394290...status code 200: 
Fetching 4424630...status code 200: 
Fetching 4573528...status code 200: 
Fetching 4437862...status code 200: 
Fetching 4427951...status code 200: 
Fetching 4385990...status code 200: 
Fetching 4369384...status code 200: 
Fetching 4393247...status code 200: 
Fetching 4379060...status code 200: 
Fetching 4442742...status code 200: 
Fetching 4407858...status code 200: 
Fetching 4465055...status code 200: 
Fetching 4437096...status code 200: 
Fetching 4458774...status code 200: 
Fetching 4506749...status code 200: 
Fetching 4428732...status code 200: 
Fetching 4411283...status code 200: 
Fetching 4520581...status code 200: 
Fetching 4451703...status code 200: 
Fetching 4482817...status code 200: 
Fetching 4462212...status code 200: 
F